In [14]:
from os import getenv
import polars as pl
from dotenv import load_dotenv
from neo4j import GraphDatabase

In [65]:
load_dotenv()

True

In [66]:
df = pl.read_csv("data/VN2018.csv")

In [67]:
df.columns

['Mã',
 'Các tỉnh thành',
 'Mật độ dân số',
 'Dân số trung bình',
 'Tỷ lệ gia tăng dân số',
 'Số trường hợp tử vong',
 'Tỷ lệ tử vong (%)',
 'Vùng',
 'Sản lượng nuôi trồng thủy sản',
 'Phần trăm sản lượng nuôi trồng thủy sản(%)',
 'Sản lượng nông nghiệp',
 'Phần trăm sản lượng nông nghiệp(%)',
 'Sản lượng lâm nghiệp',
 'Phần trăm sản lượng lâm nghiệp(%)',
 'Sản lượng phát triển tốt theo vùng ',
 'Nhiệt độ trung bình',
 'Nhiệt độ cao nhất',
 'Nhiệt độ thấp nhất',
 'Lượng mưa(mm)',
 'Tháng mùa mưa',
 'Tháng mùa khô',
 'Nhiệt độ trung bình theo vùng',
 'Lượng mưa trung bình theo vùng(mm)',
 'Tháng mưa theo vùng',
 'Tháng khô theo vùng',
 'Năm']

In [68]:
df.describe()

statistic,Mã,Các tỉnh thành,Mật độ dân số,Dân số trung bình,Tỷ lệ gia tăng dân số,Số trường hợp tử vong,Tỷ lệ tử vong (%),Vùng,Sản lượng nuôi trồng thủy sản,Phần trăm sản lượng nuôi trồng thủy sản(%),Sản lượng nông nghiệp,Phần trăm sản lượng nông nghiệp(%),Sản lượng lâm nghiệp,Phần trăm sản lượng lâm nghiệp(%),Sản lượng phát triển tốt theo vùng,Nhiệt độ trung bình,Nhiệt độ cao nhất,Nhiệt độ thấp nhất,Lượng mưa(mm),Tháng mùa mưa,Tháng mùa khô,Nhiệt độ trung bình theo vùng,Lượng mưa trung bình theo vùng(mm),Tháng mưa theo vùng,Tháng khô theo vùng,Năm
str,f64,str,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,str,str,f64,f64,str,str,f64
"""count""",63.0,"""63""",63.0,63.0,63.0,63.0,63.0,"""63""",63.0,63.0,63.0,63.0,63.0,63.0,"""63""",63.0,63.0,63.0,63.0,"""63""","""63""",63.0,63.0,"""63""","""63""",63.0
"""null_count""",0.0,"""0""",0.0,0.0,0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,"""0""","""0""",0.0,0.0,"""0""","""0""",0.0
"""mean""",49.333333,null,500.190476,1576.155556,0.954762,9037.111111,0.015868,null,66060.571429,0.015868,136687.301587,0.015878,241923.809524,0.015876,null,24.548254,36.42381,14.188889,1811.626984,null,null,24.149206,1710.31746,null,null,2018.0
"""std""",28.628882,null,634.874025,1457.499834,0.974866,6548.877693,0.011507,null,105391.49617,0.025324,141796.034218,0.016459,329184.702891,0.0216,null,1.932372,4.035876,6.756218,394.361001,null,null,1.263491,161.434779,null,null,0.0
"""min""",1.0,"""An Giang""",50.0,327.9,-1.18,1769.0,0.0031,"""Bắc Trung bộ và duyên hải miền…",408.0,0.0001,5500.0,0.0006,2000.0,0.0001,"""Lâm nghiệp""",20.0,23.7,-0.9,900.0,"""10""","""1""",22.7,1500.0,"""5,6,7,8,9,10,11""","""1""",2018.0
"""25%""",26.0,null,136.0,887.6,0.43,5768.0,0.0101,null,7124.0,0.0017,62300.0,0.0072,13700.0,0.0009,null,23.3,34.0,11.0,1550.0,null,null,23.0,1600.0,null,null,2018.0
"""50%""",48.0,null,280.0,1242.7,0.79,7421.0,0.013,null,16854.0,0.004,81100.0,0.0094,97600.0,0.0064,null,24.0,36.0,14.0,1800.0,null,null,24.7,1700.0,null,null,2018.0
"""75%""",75.0,null,689.0,1693.3,1.46,10857.0,0.0191,null,63689.0,0.0153,164500.0,0.0191,353300.0,0.0232,null,26.5,39.5,20.0,2100.0,null,null,25.1,1800.0,null,null,2018.0
"""max""",96.0,"""Đồng Tháp""",4171.0,8598.7,5.3,37014.0,0.065,"""Đồng bằng sông Hồng""",522707.0,0.1256,728500.0,0.0846,1.3132e6,0.0862,"""Thủy sản""",28.0,44.1,25.2,2926.0,"""9,11""","""2,3,4,5,6,7""",26.7,2050.0,"""9,10,11""","""12,1,2,3,4""",2018.0


In [69]:
df.head()

Mã,Các tỉnh thành,Mật độ dân số,Dân số trung bình,Tỷ lệ gia tăng dân số,Số trường hợp tử vong,Tỷ lệ tử vong (%),Vùng,Sản lượng nuôi trồng thủy sản,Phần trăm sản lượng nuôi trồng thủy sản(%),Sản lượng nông nghiệp,Phần trăm sản lượng nông nghiệp(%),Sản lượng lâm nghiệp,Phần trăm sản lượng lâm nghiệp(%),Sản lượng phát triển tốt theo vùng,Nhiệt độ trung bình,Nhiệt độ cao nhất,Nhiệt độ thấp nhất,Lượng mưa(mm),Tháng mùa mưa,Tháng mùa khô,Nhiệt độ trung bình theo vùng,Lượng mưa trung bình theo vùng(mm),Tháng mưa theo vùng,Tháng khô theo vùng,Năm
i64,str,i64,f64,f64,i64,f64,str,i64,f64,i64,f64,i64,f64,str,f64,f64,f64,f64,str,str,f64,i64,str,str,i64
1,"""Hà Nội""",2239,7520.7,2.23,37014,0.065,"""Đồng bằng sông Hồng""",102886,0.0247,196400,0.0228,31000,0.002,"""Nông nghiệp""",23.5,38.9,10.0,1676.2,"""8""","""1""",23.3,1600,"""8""","""12,1""",2018
26,"""Vĩnh Phúc""",884,1092.4,1.36,5525,0.0097,"""Đồng bằng sông Hồng""",19005,0.0046,70900,0.0082,39500,0.0026,"""Nông nghiệp""",24.25,38.5,2.0,1500.0,"""8""","""12""",23.3,1600,"""8""","""12,1""",2018
27,"""Bắc Ninh""",1516,1247.5,3.05,6346,0.0111,"""Đồng bằng sông Hồng""",35988,0.0086,68500,0.008,4200,0.0003,"""Nông nghiệp""",23.7,38.2,7.0,1575.0,"""8""","""12""",23.3,1600,"""8""","""12,1""",2018
22,"""Quảng Ninh""",205,1266.5,1.46,7145,0.0125,"""Đồng bằng sông Hồng""",56963,0.0137,46300,0.0054,376800,0.0247,"""Nông nghiệp""",22.2,36.0,7.0,1900.0,"""6,7,8""","""11, 12, 1, 2""",23.3,1600,"""8""","""12,1""",2018
30,"""Hải Dương""",1083,1807.5,1.46,10857,0.0191,"""Đồng bằng sông Hồng""",72066,0.0173,120300,0.014,3100,0.0002,"""Nông nghiệp""",23.3,33.0,15.0,1600.0,"""8""","""1""",23.3,1600,"""8""","""12,1""",2018


In [109]:
mins = df.select([pl.col(col).cast(pl.String) for col in df.columns])
df = df.with_columns(mins)

df.head()

Mã,Các tỉnh thành,Mật độ dân số,Dân số trung bình,Tỷ lệ gia tăng dân số,Số trường hợp tử vong,Tỷ lệ tử vong (%),Vùng,Sản lượng nuôi trồng thủy sản,Phần trăm sản lượng nuôi trồng thủy sản(%),Sản lượng nông nghiệp,Phần trăm sản lượng nông nghiệp(%),Sản lượng lâm nghiệp,Phần trăm sản lượng lâm nghiệp(%),Sản lượng phát triển tốt theo vùng,Nhiệt độ trung bình,Nhiệt độ cao nhất,Nhiệt độ thấp nhất,Lượng mưa(mm),Tháng mùa mưa,Tháng mùa khô,Nhiệt độ trung bình theo vùng,Lượng mưa trung bình theo vùng(mm),Tháng mưa theo vùng,Tháng khô theo vùng,Năm
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""1""","""Hà Nội""","""2239""","""7520.7""","""2.23""","""37014""","""0.065""","""Đồng bằng sông Hồng""","""102886""","""0.0247""","""196400""","""0.0228""","""31000""","""0.002""","""Nông nghiệp""","""23.5""","""38.9""","""10.0""","""1676.2""","""8""","""1""","""23.3""","""1600""","""8""","""12,1""","""2018"""
"""26""","""Vĩnh Phúc""","""884""","""1092.4""","""1.36""","""5525""","""0.0097""","""Đồng bằng sông Hồng""","""19005""","""0.0046""","""70900""","""0.0082""","""39500""","""0.0026""","""Nông nghiệp""","""24.25""","""38.5""","""2.0""","""1500.0""","""8""","""12""","""23.3""","""1600""","""8""","""12,1""","""2018"""
"""27""","""Bắc Ninh""","""1516""","""1247.5""","""3.05""","""6346""","""0.0111""","""Đồng bằng sông Hồng""","""35988""","""0.0086""","""68500""","""0.008""","""4200""","""0.0003""","""Nông nghiệp""","""23.7""","""38.2""","""7.0""","""1575.0""","""8""","""12""","""23.3""","""1600""","""8""","""12,1""","""2018"""
"""22""","""Quảng Ninh""","""205""","""1266.5""","""1.46""","""7145""","""0.0125""","""Đồng bằng sông Hồng""","""56963""","""0.0137""","""46300""","""0.0054""","""376800""","""0.0247""","""Nông nghiệp""","""22.2""","""36.0""","""7.0""","""1900.0""","""6,7,8""","""11, 12, 1, 2""","""23.3""","""1600""","""8""","""12,1""","""2018"""
"""30""","""Hải Dương""","""1083""","""1807.5""","""1.46""","""10857""","""0.0191""","""Đồng bằng sông Hồng""","""72066""","""0.0173""","""120300""","""0.014""","""3100""","""0.0002""","""Nông nghiệp""","""23.3""","""33.0""","""15.0""","""1600.0""","""8""","""1""","""23.3""","""1600""","""8""","""12,1""","""2018"""


In [ ]:
neo4j_user = getenv("NEO4J_USER")
neo4j_database = getenv("NEO4J_DB")

In [72]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=(neo4j_user, "123456789"))

if driver.verify_authentication():
    print("Authentication verified")

driver.verify_connectivity()
print("Connection verified")

Authentication verified
Connection verified


In [ ]:
def create_graph(df, driver, db, limit):
    with driver.session(database=db) as session:
        for row in df.iter_rows(named=True):
            #####
            # Tỉnh thành
            #####
            session.run(
                """
                MERGE (p:Province {name: $name, code: $code, population_density: $population_density, average_population: $average_population, growth_rate: $growth_rate, death_count: $death_count, death_rate: $death_rate, year: $year})
                """,
                name = row['Các tỉnh thành'],
                code = row['Mã'],
                population_density = row['Mật độ dân số'],
                average_population = row['Dân số trung bình'],
                growth_rate = row['Tỷ lệ gia tăng dân số'],
                death_count = row['Số trường hợp tử vong'],
                death_rate = row['Tỷ lệ tử vong (%)'],
                year = row['Năm']
            )
            
            regions = row['Vùng'].split(',')[:limit]
            avg_temperature_regions = row['Nhiệt độ trung bình theo vùng'].split(',')[:limit]
            avg_rainfall_regions = row['Lượng mưa trung bình theo vùng(mm)'].split(',')[:limit]
            rainy_months_regions = row['Tháng mưa theo vùng'].split(',')[:limit]
            dry_months_regions = row['Tháng khô theo vùng'].split(',')[:limit]       
            for region in regions:
                session.run(
                    """
                    MERGE (r: region {name: $name, avg_temperature_region: $avg_temperature_region, avg_rainfall_region: $avg_rainfall_region, rainy_months_region: $rainy_months_region, dry_months_region: $dry_months_region})
                    SET r: region
                    WITH r
                    MATCH (p: Province {code: $code})
                    MERGE (p)-[:BELONGS_TO]->(r)
                    """,
                    name = region.strip(),
                    avg_temperature_region = avg_temperature_regions[regions.index(region)].strip(),
                    avg_rainfall_region = avg_rainfall_regions[regions.index(region)].strip(),
                    rainy_months_region = rainy_months_regions[regions.index(region)].strip(),
                    dry_months_region = dry_months_regions[regions.index(region)].strip(),
                    code = row['Mã']          
                )
            
            # # Province - HAS_CLIMATE -> Climate  
            avg_temperatures = row['Nhiệt độ trung bình'].split(',')[:limit]
            highest_temperatures = row['Nhiệt độ cao nhất'].split(',')[:limit]
            lowest_temperatures = row['Nhiệt độ thấp nhất'].split(',')[:limit]  
            rainfalls = row['Lượng mưa(mm)'].split(',')[:limit]
            rainy_months = row['Tháng mùa mưa'].split(',')[:limit]
            dry_months = row['Tháng mùa khô'].split(',')[:limit]
            for avg_temperature in avg_temperatures:
                session.run(
                    """
                    MERGE (c:Climate {avg_temperature: $avg_temperature, highest_temperature: $highest_temperature, lowest_temperature: $lowest_temperature, rainfall: $rainfall, rainy_months: $rainy_months, dry_months: $dry_months})
                    SET c: Climate
                    WITH c
                    MATCH (p:Province {name: $name})
                    MERGE (p)-[:HAS_CLIMATE]->(c)
                    """,
                    avg_temperature = avg_temperature[avg_temperatures.index(avg_temperature)].strip(),
                    highest_temperature = highest_temperatures[avg_temperatures.index(avg_temperature)].strip(),
                    lowest_temperature = lowest_temperatures[avg_temperatures.index(avg_temperature)].strip(),
                    rainfall = rainfalls[avg_temperatures.index(avg_temperature)].strip(),
                    rainy_months = rainy_months[avg_temperatures.index(avg_temperature)].strip(),
                    dry_months = dry_months[avg_temperatures.index(avg_temperature)].strip(),
                    name = row['Các tỉnh thành']
                )
            
            # # Province - HAS_PRODUCTION -> Production  
            aquaculture_outputs = row['Sản lượng nuôi trồng thủy sản'].split(',')[:limit]
            agriculture_output = row['Sản lượng nông nghiệp'].split(',')[:limit]
            forestry_output = row['Sản lượng lâm nghiệp'].split(',')[:limit]
            aquaculture_ratio = row['Phần trăm sản lượng nuôi trồng thủy sản(%)'].split(',')[:limit]
            agriculture_ratio = row['Phần trăm sản lượng nông nghiệp(%)'].split(',')[:limit]
            forestry_ratio = row['Phần trăm sản lượng lâm nghiệp(%)'].split(',')[:limit]
            for aquaculture_output in aquaculture_outputs:
                session.run(
                    """
                    MERGE (sl: Production {aquaculture_output: $aquaculture_output, agriculture_output: $agriculture_output, forestry_output: $forestry_output, aquaculture_ratio: $aquaculture_ratio, agriculture_ratio: $agriculture_ratio, forestry_ratio: $forestry_ratio})
                    SET sl: Production
                    WITH sl
                    MATCH (p:Province {name: $name})
                    MERGE (p)-[:HAS_PRODUCTION]->(sl)
                    """,
                    aquaculture_output = aquaculture_output,
                    agriculture_output = agriculture_output[aquaculture_outputs.index(aquaculture_output)],
                    forestry_output = forestry_output[aquaculture_outputs.index(aquaculture_output)],
                    aquaculture_ratio = aquaculture_ratio[aquaculture_outputs.index(aquaculture_output)],
                    agriculture_ratio = agriculture_ratio[aquaculture_outputs.index(aquaculture_output)],
                    forestry_ratio = forestry_ratio[aquaculture_outputs.index(aquaculture_output)],
                    name = row['Các tỉnh thành']
                )

In [171]:
create_graph(df, driver, neo4j_database, 3)

C:\Users\Admin\AppData\Local\Temp\ipykernel_7864\2493525028.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database=db) as session:


In [172]:
driver.close()